In [109]:
def produce_random_pair(N) : 
    R = IntegerModRing(N)
    a = R.random_element()
    x = R.random_element()
    y = R.random_element()
    b = y^2 - x^3 - a*x
    disc = -16*(4*a^2+27*b^3)
    
    if disc.is_unit() == False : 
        return [gcd(ZZ(disc),N)]
        
    return [[a,b],[x,y]]


def add_points(E,P,Q) : 
    a = E[0]
    b = E[1]
    x1 = P[0]
    y1 = P[1]
    x2 = Q[0]
    y2 = Q[1]
    
    if (x2-x1).is_unit() == False :         
        raise ValueError('nonunit',x2-x1)
        
    x3 = (-x1^3 + x2*x1^2 + x2^2*x1 + y1^2 - 2*y2*y1 -x2^3 + y2^2)/(x2-x1)^2
    y3 = ((-y1 + 2*y2)*x1^3 - 3*y2*x2*x1^2 + 3*x2^2*y1*x1 + y1^3 - 3*y2*y1^2 + (-2*x2^3 + 3*y2^2)*y1 + y2*x2^3 - y2^3)\
        / (x2-x1)^3
    return [x3,y3]

def duplicate_point(E,P) :     
    a = E[0]
    b = E[1]
    x = P[0]
    y = P[1]
        
    if (2*y).is_unit() == False :
        raise ValueError('nonunit',2*y)        
        
    xx = (9*x^4+6*a*x^2-8*y^2*x+a^2)/(4*y^2)
    yy = (-27*x^6-27*a*x^4+36*y^2*x^3-9*a^2*x^2+12*a*y^2*x-8*y^4-a^3)\
         /(8*y^3)
    return [xx,yy]

def mult_point(E,P,k) : 
    
    s = Integer(k).binary() # 2-adische Entwicklung
    s = s[::-1]    # Reihenfolge der Ziffern umkehren 
    Q = 0

    for c in s :
    
        if c=='1' : 
            if Q == 0 : 
                Q = P
            else : 
                Q = add_points(E,P,Q)
        
        P = duplicate_point(E,P)            
    
    return Q

In [117]:
def Lenstra(N) :
    nKurven = 0 
    nMult = 0
    while True : 
        # Ein zufälliges Paar ([a,b],P) wird konstuiert. 
        # Dabei entspricht [a,b] einer elliptische Kurve über dem Ring Z/NZ
        # und P ist ein Punkt ub E
        ret = produce_random_pair(N)
        nKurven += 1
        
                
        if len(ret) == 1 :            
            # Falls die zufällige elliptische Kurve gar keine ist, dann 
            # ist die Diskriminante nicht invertierbar modulo N.
            if ret[0] < N :
                # Hier haben wir Glück, ein echter Teiler von N
                # wurde gefunden
                print("Primfaktor von",N,"gefunden:",ret[0],"(Variante A)")
                print("Anzahl Kurven:",nKurven)
                print("Anzahl Multiplikationen:",nVerdoppelung)
                return ret[0]
            
        if len(ret) == 2:            
            E = ret[0]
            P = ret[1]
            
            # Der Einfachheitshalber suchen bis 10!
            for k in range(1,10) : 
                try:
                    nMult += 1                    
                    P = mult_point(E,P,k)
                    
                except ValueError as err :
                    if len(err.args) != 2 or err.args[0] != 'nonunit' : 
                        break
                        
                    x = gcd(ZZ(err.args[1]),N)
                    if x < N :                                         
                        print("Primfaktor von",N,"gefunden:",x,"(Variante B)")
                        print("Anzahl Kurven:",nKurven)
                        print("Anzahl Multiplikationen:",nMult)
                        return x

            

In [118]:
p = random_prime(1000000)
q = random_prime(1000000)
N = p*q
N

637396210351

In [119]:
l = Lenstra(N)

Primfaktor von 637396210351 gefunden: 963211 (Variante B)
Anzahl Kurven: 1107
Anzahl Multiplikationen: 9961
